In [1]:
# Standard Library Imports
import json
from pathlib import Path


# Third-Party Libraries
import numpy as np
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, models, transforms

# Avalanche: Continual Learning Framework
## Benchmarks
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets.torchvision_wrapper import CIFAR10
from avalanche.benchmarks.scenarios import CLExperience
from avalanche.benchmarks.utils.flat_data import ConstantSequence

## Models
from avalanche.models import (
    MultiHeadClassifier,
    MultiTaskModule,
    MTSimpleMLP,
    MTSimpleCNN,
    PNN,
)

## Training Strategies
from avalanche.training.supervised import Naive, EWC, LwF

## Plugins and Logging
from avalanche.logging import InteractiveLogger, TextLogger
from avalanche.training.plugins import EvaluationPlugin, LRSchedulerPlugin

## Evaluation Metrics
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    forgetting_metrics,
    loss_metrics,
    timing_metrics,
    cpu_usage_metrics,
    confusion_matrix_metrics,
    disk_usage_metrics,
)

In [2]:
SAVE = False
import os

if SAVE:
    os.chdir('/home/uregina/DL_Project')
    print(os.getcwd())

# For saving the datasets/models/results/log files

if SAVE:
    DATASET_NAME = "SplitCIFAR10"
    ROOT = Path("/home/uregina/DL_Project")
    DATA_ROOT = ROOT / DATASET_NAME
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
seed = 0

DATASET_NAME = "SplitCIFAR10"
NUM_CLASSES = {
    "SplitCIFAR10": 10
}

# Define hyperparameters/scheduler/augmentation
HPARAM = {
    "batch_size": 128,        #CHANGE
    "num_epoch": 3,           #CHANGE
    "start_lr": 0.01,
    "alpha": 0.7,
    "temperature": 2,
}

In [4]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomCNN, self).__init__()

        # Adjusting the first Conv2d to take 320 channels as input (from EfficientNet b0 output)
        self.features = nn.Sequential(
            # First convolution: 320 input channels to 120 output channels (reduce parameters)
            nn.Conv2d(192, 120, kernel_size=1, stride=1, padding=0),  # 1x1 kernel reduces parameters
            nn.ReLU(inplace=True),
            # Second convolution: 128 input channels to 64 output channels (further reduce)
            nn.Conv2d(120, 64, kernel_size=1, stride=1, padding=0),  # 1x1 kernel reduces parameters
            nn.ReLU(inplace=True),
            # Ensure fixed spatial size of 1x1 using AdaptiveAvgPool2d
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Dropout(p=0.25),
        )

        # Classifier will take the 64 channels (final output from features)
        self.classifier = nn.Sequential(
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for classifier
        x = self.classifier(x)
        return x



class MTCustomCNN(CustomCNN, MultiTaskModule):
    """
    Convolutional Neural Network
    with multi-head classifier
    """

    def __init__(self):
        super().__init__()
        self.classifier = MultiHeadClassifier(64)


    def forward(self, x, task_labels):

        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the output for classifier
        x = self.classifier(x, task_labels)
        return x

from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import torch.nn as nn

class EfficientNetCNN(MultiTaskModule):
    def __init__(self):
        super(EfficientNetCNN, self).__init__()
        # Use the updated `weights` parameter instead of `pretrained`
        self.model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        
        # Freeze all parameters in the model
        for param in self.model.parameters():
            param.requires_grad = False
        
        # Extract layers up to block 7
        self.block7 = nn.Sequential(
            *list(self.model.features[:7])  # Extract up to and including block 7
        )
        
        # Custom classifier
        self.classifier = MTCustomCNN()

        # Freeze EfficientNet parameters
        for param in self.model.parameters():
            param.requires_grad = False
    
    def forward_single_task(self, x, task_id):
        x = self.block7(x)
        out = self.classifier(x, task_id)
        return out

In [5]:
import torchvision.transforms as transforms

# print to stdout
interactive_logger = InteractiveLogger()

normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )
train_transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)
test_transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]
)

benchmark = SplitCIFAR10(
    n_experiences = 5,          #CHANGE
    return_task_id = True,
    seed=seed,
    train_transform=train_transform,
    eval_transform=test_transform
)

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=False, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(
        num_classes=NUM_CLASSES[DATASET_NAME], save_image=False, stream=True
    ),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=interactive_logger,
)



Files already downloaded and verified
Files already downloaded and verified


In [6]:
MODEL_NAME = 'EfficientNetCNN'
RUN = '0'                    #Multiple runs 0,1,2
model = EfficientNetCNN()

optimizer = Adam(model.parameters(), HPARAM["start_lr"])

cl_strategy = LwF(
    model=model,
    optimizer=optimizer,
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=HPARAM["batch_size"],
    train_epochs=HPARAM["num_epoch"],
    eval_mb_size=HPARAM["batch_size"],
    alpha=HPARAM["alpha"],              # LwF parameter
    temperature=HPARAM["temperature"],  # LwF parameter
    evaluator=eval_plugin,
    device=device,
)

if SAVE:
    DATA_ROOT = ROOT / DATASET_NAME / MODEL_NAME / RUN
    DATA_ROOT.mkdir(parents=True, exist_ok=True)

In [7]:
print("Starting experiment...")
results_dict = {}  # Use a dictionary instead of a list
for index, experience in enumerate(benchmark.train_stream):
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)
    res = cl_strategy.train(experience)
    print("Training completed")
    print("Computing accuracy on the whole test")
    results_dict[index] = cl_strategy.eval(benchmark.test_stream)  # Use the index as the key

print("Experiment completed")

Starting experiment...
Start of experience:  0
Current Classes:  [1, 4]
-- >> Start of training phase << --
 10%|█         | 8/79 [00:03<00:25,  2.73it/s]

KeyboardInterrupt: 

In [ ]:
if SAVE:
    file_name = f"{MODEL_NAME}_{DATASET_NAME}_{RUN}_results.txt"
    file_path = ROOT / DATASET_NAME / MODEL_NAME / RUN / file_name
    with open(file_path, "w") as file:
        file.write(f"Model: {MODEL_NAME}\n")
        file.write(f"Dataset: {DATASET_NAME}\n")
        file.write(f"Run: {RUN}\n")  
        file.write("\nResults Dictionary:\n")
        file.write("--------------------------------------------------\n")
        for key, value in results_dict.items():
            file.write(f"Experience {key}:\n")
            for metric, metric_value in value.items():
                # Convert tensors to lists for saving
                if isinstance(metric_value, torch.Tensor):
                    metric_value = metric_value.tolist()
                file.write(f"  {metric}: {metric_value}\n")
            file.write("--------------------------------------------------\n")